In [1]:
import polars as pl
import numpy as np
import datetime  

from sklearn import preprocessing
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score

from sklearn.model_selection import train_test_split

In [2]:
#%%time
#pl.read_parquet("data/transactions_?.parquet").describe()

In [3]:
#pl.scan_parquet("data/transactions_1.parquet").collect().n_unique()

In [4]:
delweek=93
delweek=105
delweek=100

In [5]:
workdays={}
d1 = datetime.date(2022, 1, 1)
while d1<datetime.date(2025, 1, 1):
    #print(d1)
    #workdays.append([d1,0 if d1.weekday() in [5,6] else 1 ] )
    workdays[d1] = 0 if d1.weekday() in [5,6] else 1
    
    d1=d1+datetime.timedelta(1) 
workdays[datetime.date(2022, 1, 3)] = 0
workdays[datetime.date(2022, 1, 4)] = 0
workdays[datetime.date(2022, 1, 5)] = 0
workdays[datetime.date(2022, 1, 6)] = 0
workdays[datetime.date(2022, 1, 7)] = 0

workdays[datetime.date(2022, 2, 23)] = 0

workdays[datetime.date(2022, 3, 5)] = 1
workdays[datetime.date(2022, 3, 7)] = 0
workdays[datetime.date(2022, 3, 8)] = 0

workdays[datetime.date(2022, 5, 2)] = 0
workdays[datetime.date(2022, 5, 3)] = 0
workdays[datetime.date(2022, 5, 9)] = 0
workdays[datetime.date(2022, 5, 10)] = 0

workdays[datetime.date(2022, 6, 13)] = 0

workdays[datetime.date(2022, 11, 4)] = 0
########################################## 2023
workdays[datetime.date(2023, 1, 2)] = 0
workdays[datetime.date(2023, 1, 3)] = 0
workdays[datetime.date(2023, 1, 4)] = 0
workdays[datetime.date(2023, 1, 5)] = 0
workdays[datetime.date(2023, 1, 6)] = 0

workdays[datetime.date(2023, 2, 23)] = 0
workdays[datetime.date(2023, 2, 24)] = 0

workdays[datetime.date(2023, 3, 8)] = 0

workdays[datetime.date(2023, 5, 1)] = 0
workdays[datetime.date(2023, 5, 8)] = 0
workdays[datetime.date(2023, 5, 9)] = 0

workdays[datetime.date(2023, 6, 12)] = 0

workdays[datetime.date(2023, 11, 6)] = 0
########################################## 2024
workdays[datetime.date(2024, 1, 1)] = 0
workdays[datetime.date(2024, 1, 2)] = 0
workdays[datetime.date(2024, 1, 3)] = 0
workdays[datetime.date(2024, 1, 6)] = 0
workdays[datetime.date(2024, 1, 7)] = 0
workdays[datetime.date(2024, 1, 8)] = 0

workdays[datetime.date(2024, 5, 1)] = 0
workdays[datetime.date(2024, 5, 2)] = 0
workdays[datetime.date(2024, 5, 8)] = 0
workdays[datetime.date(2024, 5, 9)] = 0

workdays[datetime.date(2024, 6, 12)] = 0
workdays[datetime.date(2024, 6, 13)] = 0

workdays[datetime.date(2024, 11, 1)] = 1
workdays[datetime.date(2024, 11, 3)] = 0
workdays[datetime.date(2024, 11, 4)] = 0

workdays[datetime.date(2024, 12, 31)] = 0

workdays

calendar000 = pl.read_csv("data/calendar.csv", try_parse_dates=True)
calendar000.with_columns(
                      (pl.col("date")+datetime.timedelta(-5)).replace(workdays).cast(pl.Int8).alias("work_5"), 
                      (pl.col("date")+datetime.timedelta(-4)).replace(workdays).cast(pl.Int8).alias("work_4"), 
                      (pl.col("date")+datetime.timedelta(-3)).replace(workdays).cast(pl.Int8).alias("work_3"), 
                      (pl.col("date")+datetime.timedelta(-2)).replace(workdays).cast(pl.Int8).alias("work_2"), 
                      (pl.col("date")+datetime.timedelta(-1)).replace(workdays).cast(pl.Int8).alias("work_1"), 
                      pl.col("date").replace(workdays).cast(pl.Int8).alias("work"),
                      (pl.col("date")+datetime.timedelta(1)).replace(workdays).cast(pl.Int8).alias("work1"), 
                      (pl.col("date")+datetime.timedelta(2)).replace(workdays).cast(pl.Int8).alias("work2"), 
                      (pl.col("date")+datetime.timedelta(3)).replace(workdays).cast(pl.Int8).alias("work3"), 
                      (pl.col("date")+datetime.timedelta(4)).replace(workdays).cast(pl.Int8).alias("work4"), 
                      (pl.col("date")+datetime.timedelta(5)).replace(workdays).cast(pl.Int8).alias("work5"), 
                     ).with_columns( 
                      (pl.col("work1")+pl.col("work2")+pl.col("work3")+pl.col("work4")+pl.col("work5")).alias("w5_after"), 
                      (pl.col("work1")+pl.col("work2")+pl.col("work3")+pl.col("work4")).alias("w4_after"), 
                      (pl.col("work1")+pl.col("work2")+pl.col("work3")).alias("w3_after"), 
                      (pl.col("work1")+pl.col("work2")).alias("w2_after"), 
                      (pl.col("work_1")+pl.col("work_2")).alias("w2_before"), 
                      (pl.col("work_1")+pl.col("work_2")+pl.col("work_3")).alias("w3_before"), 
                      (pl.col("work_1")+pl.col("work_2")+pl.col("work_3")+pl.col("work_4")).alias("w4_before"), 
                      (pl.col("work_1")+pl.col("work_2")+pl.col("work_3")+pl.col("work_4")+pl.col("work_5")).alias("w5_before"), 
                     )

date,week,part,work_5,work_4,work_3,work_2,work_1,work,work1,work2,work3,work4,work5,w5_after,w4_after,w3_after,w2_after,w2_before,w3_before,w4_before,w5_before
date,i64,str,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8
2022-07-25,0,"""train""",1,1,1,0,0,1,1,1,1,1,0,4,4,3,2,0,1,2,3
2022-07-26,0,"""train""",1,1,0,0,1,1,1,1,1,0,0,3,3,3,2,1,1,2,3
2022-07-27,0,"""train""",1,0,0,1,1,1,1,1,0,0,1,3,2,2,2,2,2,2,3
2022-07-28,0,"""train""",0,0,1,1,1,1,1,0,0,1,1,3,2,1,1,2,3,3,3
2022-07-29,0,"""train""",0,1,1,1,1,1,0,0,1,1,1,3,2,1,0,2,3,4,4
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024-10-23,117,"""private""",1,0,0,1,1,1,1,1,0,0,1,3,2,2,2,2,2,2,3
2024-10-24,117,"""private""",0,0,1,1,1,1,1,0,0,1,1,3,2,1,1,2,3,3,3
2024-10-25,117,"""private""",0,1,1,1,1,1,0,0,1,1,1,3,2,1,0,2,3,4,4


In [6]:
%%time
innstat000 = pl.read_parquet("data/target_series.parquet"
                            ).with_columns(                              
                                pl.col("week").cast(pl.Int8),
                            ).filter( pl.col("week")<delweek 
                            ).filter( pl.col("target")>0 
                            ).with_columns(                              
                                (pl.col("week")-pl.col("week").shift(1).over("inn_id")).alias("lbreak"),
                                    ).group_by("inn_id"
                                              ).agg((delweek-pl.col("week").max()-1).alias("endbreak"),
                                                    (pl.col("week").max()-pl.col("week").min()+1).alias("length"),
                                                    #pl.col("week").min().alias("startweek"),
                                                    pl.col("week").count().alias("countpositive"),
                                                    pl.col("lbreak").max().alias("maxlbreak"),
                                                   ).with_columns((pl.col("length")-pl.col("countpositive")).alias("countzero") 
                                                                 )
innstat000#.max()

CPU times: user 838 ms, sys: 521 ms, total: 1.36 s
Wall time: 750 ms


inn_id,endbreak,length,countpositive,maxlbreak,countzero
str,i8,i8,u32,i8,i64
"""inn1958397""",0,100,88,2,12
"""inn4309075""",0,100,100,1,0
"""inn4380818""",0,100,99,2,1
"""inn5142015""",0,76,57,14,19
"""inn1277159""",0,100,99,2,1
…,…,…,…,…,…
"""inn1133791""",0,67,67,1,0
"""inn1431658""",0,100,100,1,0
"""inn1737624""",0,100,95,5,5


In [7]:
%%time
innweeks = pl.read_parquet("data/target_series.parquet"
                            ).with_columns(                              
                                pl.col("week").cast(pl.Int8),
                                np.log1p(pl.col("target")),
                            ).with_columns(                              
                                np.sign(pl.col("target")).cast(pl.Float32).alias("sign"),
                            )
innweeks#.filter( pl.col("target")<5).filter( pl.col("target")>0)#.sum()

CPU times: user 281 ms, sys: 117 ms, total: 398 ms
Wall time: 192 ms


inn_id,week,target,sign
str,i8,f64,f32
"""inn1000051""",0,12.95314,1.0
"""inn1000051""",1,7.392578,1.0
"""inn1000051""",2,13.929488,1.0
"""inn1000051""",3,10.583951,1.0
"""inn1000051""",4,12.972155,1.0
…,…,…,…
"""inn999886""",101,15.637834,1.0
"""inn999886""",102,17.042798,1.0
"""inn999886""",103,16.736929,1.0


In [8]:
#innweeks.group_by("inn_id").agg( pl.col("week").count()-pl.col("week").max()+pl.col("week").min() )["week"].value_counts()

In [9]:
%%time
innweeks = innweeks.with_columns(
                      #pl.col("week").rolling_min(window_size=53,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("lengthweek"),
                      #pl.col("sign").rolling_sum(window_size=53,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active53"),
                      #pl.col("sign").rolling_sum(window_size=39,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active39"),
                      #pl.col("sign").rolling_sum(window_size=26,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active26"),
                      #pl.col("sign").rolling_sum(window_size=16,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active16"),
                      #pl.col("sign").rolling_sum(window_size=15,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active15"),
                      #pl.col("sign").rolling_sum(window_size=14,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active14"),
                      #pl.col("sign").rolling_sum(window_size=13,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active13"),
                      #pl.col("sign").rolling_sum(window_size=12,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active12"),
                      #pl.col("sign").rolling_sum(window_size=11,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active11"),
                      #pl.col("sign").rolling_sum(window_size=10,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active10"),
                      #pl.col("sign").rolling_sum(window_size=9,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active9"),
                      #pl.col("sign").rolling_sum(window_size=8,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active8"),
                      #pl.col("sign").rolling_sum(window_size=7,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active7"),
                      #pl.col("sign").rolling_sum(window_size=6,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active6"),
                      #pl.col("sign").rolling_sum(window_size=5,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active5"),
                      #pl.col("sign").rolling_sum(window_size=4,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active4"),
                      #pl.col("sign").rolling_sum(window_size=3,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active3"),
                      #pl.col("sign").rolling_sum(window_size=2,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active2"),
                      #pl.col("sign").rolling_sum(window_size=1,min_periods=1).cast(pl.Int8).shift(1).over("inn_id").alias("active1"),
                      pl.col("target").shift(-1).over("inn_id").alias("target_1"),
                      pl.col("target").shift(-2).over("inn_id").alias("target_2"),
                      pl.col("target").shift(-3).over("inn_id").alias("target_3"),
                   
                      pl.col("target").shift(1).over("inn_id").alias("imax1"),
                      pl.col("target").shift(2).over("inn_id").alias("imax2"),
                      pl.col("target").shift(3).over("inn_id").alias("imax3"),
                      pl.col("target").shift(4).over("inn_id").alias("imax4"),
                      pl.col("target").shift(5).over("inn_id").alias("imax5"),
                      pl.col("target").shift(6).over("inn_id").alias("imax6"),
                      pl.col("target").shift(7).over("inn_id").alias("imax7"),
                      pl.col("target").shift(8).over("inn_id").alias("imax8"),
                      pl.col("target").shift(9).over("inn_id").alias("imax9"),
                      pl.col("target").shift(10).over("inn_id").alias("imax10"),
                      pl.col("target").shift(11).over("inn_id").alias("imax11"),
                      pl.col("target").shift(12).over("inn_id").alias("imax12"),
                      pl.col("target").shift(13).over("inn_id").alias("imax13"),
                      pl.col("target").shift(14).over("inn_id").alias("imax14"),
                      pl.col("target").shift(15).over("inn_id").alias("imax15"),
                      pl.col("target").shift(16).over("inn_id").alias("imax16"),
                      pl.col("target").shift(17).over("inn_id").alias("imax17"),
                      pl.col("target").shift(18).over("inn_id").alias("imax18"),
                      pl.col("target").shift(19).over("inn_id").alias("imax19"),
                      pl.col("target").shift(20).over("inn_id").alias("imax20"),
                      pl.col("target").shift(21).over("inn_id").alias("imax21"),
                      pl.col("target").shift(22).over("inn_id").alias("imax22"),
                      pl.col("target").shift(23).over("inn_id").alias("imax23"),
                      pl.col("target").shift(24).over("inn_id").alias("imax24"),
                      pl.col("target").shift(25).over("inn_id").alias("imax25"),
                      pl.col("target").shift(26).over("inn_id").alias("imax26"),
                      pl.col("target").shift(27).over("inn_id").alias("imax27"),
                      pl.col("target").shift(28).over("inn_id").alias("imax28"),
                      pl.col("target").shift(29).over("inn_id").alias("imax29"),
                      pl.col("target").shift(30).over("inn_id").alias("imax30"),
                      pl.col("target").shift(31).over("inn_id").alias("imax31"),
                      pl.col("target").shift(32).over("inn_id").alias("imax32"),
                      pl.col("target").shift(33).over("inn_id").alias("imax33"),
                      pl.col("target").shift(34).over("inn_id").alias("imax34"),
                      pl.col("target").shift(35).over("inn_id").alias("imax35"),
                      pl.col("target").shift(36).over("inn_id").alias("imax36"),
                      pl.col("target").shift(37).over("inn_id").alias("imax37"),
                      pl.col("target").shift(38).over("inn_id").alias("imax38"),
                      pl.col("target").shift(39).over("inn_id").alias("imax39"),
                      pl.col("target").shift(40).over("inn_id").alias("imax40"),
                      pl.col("target").shift(41).over("inn_id").alias("imax41"),
                      pl.col("target").shift(42).over("inn_id").alias("imax42"),
                      pl.col("target").shift(43).over("inn_id").alias("imax43"),
                      pl.col("target").shift(44).over("inn_id").alias("imax44"),
                      pl.col("target").shift(45).over("inn_id").alias("imax45"),
                      pl.col("target").shift(46).over("inn_id").alias("imax46"),
                      pl.col("target").shift(47).over("inn_id").alias("imax47"),
                      pl.col("target").shift(48).over("inn_id").alias("imax48"),
                      pl.col("target").shift(49).over("inn_id").alias("imax49"),
                      pl.col("target").shift(50).over("inn_id").alias("imax50"),
                      pl.col("target").shift(51).over("inn_id").alias("imax51"),
                      pl.col("target").shift(52).over("inn_id").alias("imax52"),
                      pl.col("target").shift(53).over("inn_id").alias("imax53"),
                      pl.col("target").shift(54).over("inn_id").alias("imax54"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(55).over("inn_id").alias("imax55"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(56).over("inn_id").alias("imax56"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(57).over("inn_id").alias("imax57"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(58).over("inn_id").alias("imax58"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(59).over("inn_id").alias("imax59"),
                      
                      #pl.col("target").rolling_max(window_size=53,min_periods=1).shift(1).over("inn_id").alias("imax"),
                      #pl.col("target").rolling_min(window_size=53,min_periods=1).shift(1).over("inn_id").alias("imin"),
                      pl.col("target").rolling_mean(window_size=53,min_periods=1).shift(1).over("inn_id").alias("imean"),
                      pl.col("target").rolling_median(window_size=53,min_periods=1).shift(1).over("inn_id").alias("imedian"),
    
                      #pl.col("target").rolling_quantile(quantile=0.95, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq095"),
                      #pl.col("target").rolling_quantile(quantile=0.90, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq090"),
                      #pl.col("target").rolling_quantile(quantile=0.85, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq085"),
                      #pl.col("target").rolling_quantile(quantile=0.80, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq080"),
                      #pl.col("target").rolling_quantile(quantile=0.75, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq075"),
                      #pl.col("target").rolling_quantile(quantile=0.70, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq070"),
                      #pl.col("target").rolling_quantile(quantile=0.65, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq065"),
                      #pl.col("target").rolling_quantile(quantile=0.60, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq060"),
                      #pl.col("target").rolling_quantile(quantile=0.55, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq055"),
    
                      #pl.col("target").rolling_quantile(quantile=0.45, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq045"),
                      #pl.col("target").rolling_quantile(quantile=0.40, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq040"),
                      #pl.col("target").rolling_quantile(quantile=0.35, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq035"),
                      #pl.col("target").rolling_quantile(quantile=0.30, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq030"),
                      #pl.col("target").rolling_quantile(quantile=0.25, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq025"),
                      #pl.col("target").rolling_quantile(quantile=0.20, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq020"),
                      #pl.col("target").rolling_quantile(quantile=0.15, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq015"),
                      #pl.col("target").rolling_quantile(quantile=0.10, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq010"),
                      #pl.col("target").rolling_quantile(quantile=0.05, window_size=53,min_periods=1).shift(1).over("inn_id").alias("iq005"),
                     )
innweeks

CPU times: user 11.8 s, sys: 7.64 s, total: 19.4 s
Wall time: 18.9 s


inn_id,week,target,sign,target_1,target_2,target_3,imax1,imax2,imax3,imax4,imax5,imax6,imax7,imax8,imax9,imax10,imax11,imax12,imax13,imax14,imax15,imax16,imax17,imax18,imax19,imax20,imax21,imax22,imax23,imax24,imax25,imax26,imax27,imax28,imax29,imax30,imax31,imax32,imax33,imax34,imax35,imax36,imax37,imax38,imax39,imax40,imax41,imax42,imax43,imax44,imax45,imax46,imax47,imax48,imax49,imax50,imax51,imax52,imax53,imax54,imean,imedian
str,i8,f64,f32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""inn1000051""",0,12.95314,1.0,7.392578,13.929488,10.583951,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""inn1000051""",1,7.392578,1.0,13.929488,10.583951,12.972155,12.95314,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,12.95314,12.95314
"""inn1000051""",2,13.929488,1.0,10.583951,12.972155,0.0,7.392578,12.95314,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,10.172859,10.172859
"""inn1000051""",3,10.583951,1.0,12.972155,0.0,13.739822,13.929488,7.392578,12.95314,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,11.425069,12.95314
"""inn1000051""",4,12.972155,1.0,0.0,13.739822,14.26319,10.583951,13.929488,7.392578,12.95314,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,11.214789,11.768546
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""inn999886""",101,15.637834,1.0,17.042798,16.736929,17.025046,16.703016,16.853691,16.48522,16.193706,16.48499,16.786354,16.414182,15.974485,16.120146,16.788546,16.801483,15.919157,15.546629,15.973451,16.08884,16.104969,15.932649,16.509635,16.137412,17.231515,15.362628,15.992497,16.084429,16.099601,14.700636,11.500407,16.350607,16.26866,15.943668,16.089237,15.602697,15.899565,16.653305,16.789835,16.902591,16.199541,15.998248,16.52875,16.269302,16.993851,16.649667,16.347232,16.586483,14.772085,14.507382,16.361285,16.11918,15.040814,16.209546,16.1878,16.785001,16.540138,16.088292,16.577502,16.103303,16.193706
"""inn999886""",102,17.042798,1.0,16.736929,17.025046,16.799984,15.637834,16.703016,16.853691,16.48522,16.193706,16.48499,16.786354,16.414182,15.974485,16.120146,16.788546,16.801483,15.919157,15.546629,15.973451,16.08884,16.104969,15.932649,16.509635,16.137412,17.231515,15.362628,15.992497,16.084429,16.099601,14.700636,11.500407,16.350607,16.26866,15.943668,16.089237,15.602697,15.899565,16.653305,16.789835,16.902591,16.199541,15.998248,16.52875,16.269302,16.993851,16.649667,16.347232,16.586483,14.772085,14.507382,16.361285,16.11918,15.040814,16.209546,16.1878,16.785001,16.540138,16.088292,16.094803,16.193706
"""inn999886""",103,16.736929,1.0,17.025046,16.799984,null,17.042798,15.637834,16.703016,16.853691,16.48522,16.193706,16.48499,16.786354,16.414182,15.974485,16.120146,16.788546,16.8014

In [10]:
calendar = pl.read_csv("data/calendar.csv", try_parse_dates=True
                      ).with_columns(
                              pl.col("week").cast(pl.Int8),
)
calendar

date,week,part
date,i8,str
2022-07-25,0,"""train"""
2022-07-26,0,"""train"""
2022-07-27,0,"""train"""
2022-07-28,0,"""train"""
2022-07-29,0,"""train"""
…,…,…
2024-10-23,117,"""private"""
2024-10-24,117,"""private"""
2024-10-25,117,"""private"""


In [11]:
weeks = calendar.group_by("week").agg(
                              pl.col("date").min().alias("d1"),
                              pl.col("date").max().alias("d2"),
                             ).with_columns(
                              pl.col("week").cast(pl.Int8),
    
                              pl.col("d1").dt.year().cast(pl.Int16).alias("year1"),
                              pl.col("d1").dt.month().alias("month1"),
                              pl.col("d1").dt.day().alias("day1"),
                              pl.col("d1").dt.ordinal_day().alias("dy1"),
    
                              pl.col("d2").dt.year().cast(pl.Int16).alias("year2"),
                              pl.col("d2").dt.month().alias("month2"),
                              pl.col("d2").dt.day().alias("day2"),
                              pl.col("d2").dt.ordinal_day().alias("dy2"),
                             ).with_columns(
                              ((pl.col("month2")-pl.col("month1")+12)%12).alias("m"),
                              (pl.col("year2")-pl.col("year1")).cast(pl.Int8).alias("y"),
    
                              pl.when( (pl.col("month1")>2)&(pl.col("year1")%4!=0)).then(pl.col("dy1")+1).otherwise(pl.col("dy1")).alias("dy1"),
                              pl.when( (pl.col("month2")>2)&(pl.col("year2")%4!=0)).then(pl.col("dy2")+1).otherwise(pl.col("dy2")).alias("dy2"),
                              #(pl.col("d1")+datetime.timedelta(-1)).replace(workdays).cast(pl.Int8).alias("work_1"), 
                              (pl.col("d1")+datetime.timedelta(0)).replace(workdays).cast(pl.Int8).alias("work1"), 
                              (pl.col("d1")+datetime.timedelta(1)).replace(workdays).cast(pl.Int8).alias("work2"), 
                              (pl.col("d1")+datetime.timedelta(2)).replace(workdays).cast(pl.Int8).alias("work3"), 
                              (pl.col("d1")+datetime.timedelta(3)).replace(workdays).cast(pl.Int8).alias("work4"), 
                              (pl.col("d1")+datetime.timedelta(4)).replace(workdays).cast(pl.Int8).alias("work5"), 
                              (pl.col("d1")+datetime.timedelta(5)).replace(workdays).cast(pl.Int8).alias("work6"), 
                              (pl.col("d1")+datetime.timedelta(6)).replace(workdays).cast(pl.Int8).alias("work7"), 
                              #(pl.col("d1")+datetime.timedelta(7)).replace(workdays).cast(pl.Int8).alias("work8"), 
                             ).with_columns( 
                              (pl.col("work1")+pl.col("work2")+pl.col("work3")+pl.col("work4")+pl.col("work5")+pl.col("work6")+pl.col("work7")).alias("w7"), 
                             ).with_columns( 
                              pl.col("w7").shift(1).alias("w7_1"), 
                              pl.col("w7").shift(2).alias("w7_2"), 
                              pl.col("w7").shift(3).alias("w7_3"), 
                              pl.col("w7").shift(4).alias("w7_4"), 
                              pl.col("w7").shift(5).alias("w7_5"), 
                              pl.col("w7").shift(6).alias("w7_6"), 
                             )
weeks

week,d1,d2,year1,month1,day1,dy1,year2,month2,day2,dy2,m,y,work1,work2,work3,work4,work5,work6,work7,w7,w7_1,w7_2,w7_3,w7_4,w7_5,w7_6
i8,date,date,i16,i8,i8,i16,i16,i8,i8,i16,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8
88,2024-04-01,2024-04-07,2024,4,1,92,2024,4,7,98,0,0,1,1,1,1,1,0,0,5,null,null,null,null,null,null
54,2023-08-07,2023-08-13,2023,8,7,220,2023,8,13,226,0,0,1,1,1,1,1,0,0,5,5,null,null,null,null,null
6,2022-09-05,2022-09-11,2022,9,5,249,2022,9,11,255,0,0,1,1,1,1,1,0,0,5,5,5,null,null,null,null
37,2023-04-10,2023-04-16,2023,4,10,101,2023,4,16,107,0,0,1,1,1,1,1,0,0,5,5,5,5,null,null,null
25,2023-01-16,2023-01-22,2023,1,16,16,2023,1,22,22,0,0,1,1,1,1,1,0,0,5,5,5,5,5,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
60,2023-09-18,2023-09-24,2023,9,18,262,2023,9,24,268,0,0,1,1,1,1,1,0,0,5,2,3,5,5,5,5
104,2024-07-22,2024-07-28,2024,7,22,204,2024,7,28,210,0,0,1,1,1,1,1,0,0,5,5,2,3,5,5,5
38,2023-04-17,2023-04-23,2023,4,17,108,2023,4,23,114,0,0,1,1,1,1,1,0,0,5,5,5,2,3,5,5


In [12]:
profiles = pl.read_parquet("data/profiles.parquet")#["diff_datopen_report_date_flg"].unique()
age={
    "1m":1,
    "2_3m":3,
    "3_6m":6,
    "6_12m":12,
    "1_2y":24,
    "2_4y":48,
    "4_8y":96,
    "8_12y":144,
    "more_12y":192,
    #"NULL":np.nan
}    
ipul={
    "ip":0,
    "ul":1,
}    
profiles = profiles.with_columns(
    pl.col("diff_datopen_report_date_flg").replace(age, default=np.NAN),
    pl.col("ipul").replace(ipul).cast(pl.String),
    pl.col("id_region").fill_null("-"),
    pl.col("main_okved_group").fill_null("-"),
)
profiles = profiles.sort("report_date"
                        ).group_by("inn_id"
                                  ).agg(pl.col("ipul").first(),
                                        pl.col("id_region").first(),
                                        pl.col("main_okved_group").first(),
                                        pl.col("diff_datopen_report_date_flg").first(),
                                       )
profiles

/tmp/ipykernel_59600/3623493460.py:19: DeprecationWarning: The `default` parameter for `replace` is deprecated. Use `replace_strict` instead to set a default while replacing values.
  pl.col("diff_datopen_report_date_flg").replace(age, default=np.NAN),


inn_id,ipul,id_region,main_okved_group,diff_datopen_report_date_flg
str,str,str,str,f64
"""inn3506991""","""0""","""40""","""52""",192.0
"""inn1819197""","""1""","""78""","""85""",192.0
"""inn970848""","""0""","""46""","""43""",48.0
"""inn5052307""","""1""","""7""","""47""",144.0
"""inn355475""","""0""","""3""","""62""",144.0
…,…,…,…,…
"""inn2297095""","""0""","""46""","""81""",6.0
"""inn3375424""","""0""","""92""","""68""",192.0
"""inn4511294""","""1""","""92""","""43""",96.0


In [13]:
XX = innweeks.join(weeks,
                   on="week",
                   how="left"
                  ).join(profiles,on="inn_id",
                         how="left"
                        ).sort("week"
                                ) #.filter(pl.col("week")>4)
XX

inn_id,week,target,sign,target_1,target_2,target_3,imax1,imax2,imax3,imax4,imax5,imax6,imax7,imax8,imax9,imax10,imax11,imax12,imax13,imax14,imax15,imax16,imax17,imax18,imax19,imax20,imax21,imax22,imax23,imax24,imax25,imax26,imax27,imax28,imax29,imax30,…,imax50,imax51,imax52,imax53,imax54,imean,imedian,d1,d2,year1,month1,day1,dy1,year2,month2,day2,dy2,m,y,work1,work2,work3,work4,work5,work6,work7,w7,w7_1,w7_2,w7_3,w7_4,w7_5,w7_6,ipul,id_region,main_okved_group,diff_datopen_report_date_flg
str,i8,f64,f32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,date,date,i16,i8,i8,i16,i16,i8,i8,i16,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,str,str,str,f64
"""inn1000051""",0,12.95314,1.0,7.392578,13.929488,10.583951,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,2022-07-25,2022-07-31,2022,7,25,207,2022,7,31,213,0,0,1,1,1,1,1,0,0,5,5,5,5,5,5,4,"""0""","""45""","""68""",48.0
"""inn1000246""",0,13.911296,1.0,13.679488,13.335954,12.630267,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,2022-07-25,2022-07-31,2022,7,25,207,2022,7,31,213,0,0,1,1,1,1,1,0,0,5,5,5,5,5,5,4,"""1""","""33""","""17""",96.0
"""inn1000281""",0,0.0,0.0,0.0,10.467411,11.994005,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,2022-07-25,2022-07-31,2022,7,25,207,2022,7,31,213,0,0,1,1,1,1,1,0,0,5,5,5,5,5,5,4,"""1""","""40""","""20""",192.0
"""inn1000367""",0,12.620393,1.0,0.0,8.08754,12.917566,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,2022-07-25,2022-07-31,2022,7,25,207,2022,7,31,213,0,0,1,1,1,1,1,0,0,5,5,5,5,5,5,4,"""1""","""41""","""25""",96.0
"""inn1000409""",0,12.576319,1.0,11.939092,11.059709,10.855669,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,2022-07-25,2022-07-31,2022,7,25,207,2022,7,31,213,0,0,1,1,1,1,1,0,0,5,5,5,5,5,5,4,"""0""","""45""","""68""",192.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""inn999585""",105,15.368828,1.0,null,null,null,15.710406,15.39354,16.016178,15.961055,15.256314,15.697546,15.585403,15.705385,16.071476,15.752464,15.260163,15.586704,15.373119,15.536085,16.221475,15.510999,15.310495,16.296947,16.141412,15.407971,15.725966,15.133534,15.858813,15.619837,15.628901,15.658263,13.111548,15.253346,15.796855,9.662077,…,15.631087,15.723983,15.239363,15.26733,16.023047,15.378676,15.619837,2024-07-29,2024-08-04,2024,7,29,211,2024,8,4,217,1,0,1,1,1,1,1,0,0,5,5,5,5,5,5,5,"""1""","""70""","""35""",192.0
"""inn999624""",105,12.198977,1.0,null,null,null,12.699588,12.93888,12.295488,12.524511,12.45583,12.56929,12.003098,12.254917,12.295468,12.434921,12.210596,10.767151,12.615497,12.068944,11.706759,12.133861,12.084767,11.79904,11.769885,11.971913,11.498465,11.830561,12.047565,11.353543,12.100351,12.285908,11.902995,11.618633,11.376415,12.17889,…,12.624183,13.361284,12.830919,11.366589,12.837353,12.130665,12.210596,2024-07-29,2024-08-04,2024,7,29,211,2024,8,4,217,1,0,1,1,1,1,1,0,0,5,5,5,5,5,5,5,"""0""","""46""","""47""",192.0
"""inn999737""",105,0.0,0.0,null,null,null,0.0,0.0,0.0,15.371876,12.877789,13.456792,13.15064,13.269197,0.0,13.85753,12.889064,12.031296,11.246562,13.616818,15.311301,11.52302,12.022291,12.04368,13.992193,14.181685,14.01396

In [14]:
y = XX.select("target","target_1","target_2","target_3").to_numpy()
X = XX.drop(["target","target_1","target_2","target_3",
             "sign",
             "d1",
             "d2",
             #"week",
             #"inn_id",
             #"report_date"
            ]).to_pandas()

border=delweek
X_train = X[X["week"]<border].drop("week",axis=1) 
y_train = y[X["week"]<border]    


X_test = X[X["week"].isin(range(border,103))].drop("week",axis=1) 
y_test = y[X["week"].isin(range(border,103))] 
X_test

,inn_id,imax1,imax2,imax3,imax4,imax5,imax6,imax7,imax8,imax9,...,w7_1,w7_2,w7_3,w7_4,w7_5,w7_6,ipul,id_region,main_okved_group,diff_datopen_report_date_flg
5196300,inn1000051,10.373773,9.118413,13.542485,13.573879,14.297409,11.258165,9.807153,11.321931,12.904122,...,5.0,5.0,5.0,2.0,3.0,5.0,0,45,68,48.0
5196301,inn1000246,15.219413,14.639067,14.987862,14.969566,14.869111,15.179490,14.362012,15.241162,15.032990,...,5.0,5.0,5.0,2.0,3.0,5.0,1,33,17,96.0
5196302,inn1000281,11.652865,11.490739,11.325688,13.429049,13.287213,12.834105,13.477381,0.000000,11.455340,...,5.0,5.0,5.0,2.0,3.0,5.0,1,40,20,192.0
5196303,inn1000367,13.394811,13.689302,13.887568,14.189779,13.274840,13.501493,13.467146,13.624901,13.584532,...,5.0,5.0,5.0,2.0,3.0,5.0,1,41,25,96.0
5196304,inn1000409,12.403199,9.961868,10.120735,11.971783,7.970205,13.080359,0.000000,0.000000,12.091830,...,5.0,5.0,5.0,2.0,3.0,5.0,0,45,68,192.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5352184,inn999585,15.961055,15.256314,15.697546,15.585403,15.705385,16.071476,15.752464,15.260163,15.586704,...,5.0,5.0,5.0,5.0,5.0,5.0,1,70,35,192.0
5352185,inn999624,12.524511,12.455830,12.569290,12.003098,12.254917,12.295468,12.434921,12.210596,10.767151,...,5.0,5.0,5.0,5.0,5.0,5.0,0,46,47,192.0
5352186,inn999737,15.371876,12.877789,13.456792,13.150640,13.269197,0.000000,13.857530,12.889064,12.031296,...,5.0,5.0,5.0,5.0,5.0,5.0,1,46,22,96.0
5352187,inn999759,15.891348,16.081322,16.312722,16.374579,16.790556,16.049659,16.215580,16.167412,15.183374,...,5.0,5.0,5.0,5.0,5.0,5.0,1,70,25,144.0


In [15]:
model = CatBoostRegressor(
                            loss_function='MultiRMSE', 

                            depth=15,
                            cat_features=["inn_id",
                                          "ipul",
                                          "id_region",
                                          "main_okved_group",
                                          #"report_date"
                                         ],
                            one_hot_max_size=256,
                            #l2_leaf_reg=3,
                            task_type="GPU",
                            model_size_reg=10,
                            random_seed=0,
                            has_time=True,
                         )
model.fit(X_train,y_train,
          eval_set=(X_test, y_test),
          plot=True,
          verbose=100,
          use_best_model=True,
          early_stopping_rounds=30,
         )
#model.save_model(f"model{model.learning_rate_}_{model.best_score_}.cbm")
#bestTest = 1.894337907
#bestTest = 1.877237918
#bestTest = 1.410811804 1,46
#bestTest = 4.411957296
#bestTest = 4.410412236

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 7.8538677	test: 6.3494683	best: 6.3494683 (0)	total: 908ms	remaining: 15m 6s
100:	learn: 5.0842574	test: 4.4725585	best: 4.4725585 (100)	total: 1m 29s	remaining: 13m 15s
200:	learn: 4.9857747	test: 4.4039397	best: 4.4039397 (200)	total: 3m 13s	remaining: 12m 50s
300:	learn: 4.9158617	test: 4.3868449	best: 4.3868449 (300)	total: 5m 2s	remaining: 11m 41s
400:	learn: 4.8506331	test: 4.3761073	best: 4.3761073 (400)	total: 6m 57s	remaining: 10m 23s
500:	learn: 4.7977158	test: 4.3699412	best: 4.3699157 (498)	total: 8m 51s	remaining: 8m 49s
600:	learn: 4.7490195	test: 4.3655596	best: 4.3655596 (600)	total: 10m 44s	remaining: 7m 7s
700:	learn: 4.7096375	test: 4.3622643	best: 4.3622643 (700)	total: 12m 32s	remaining: 5m 20s
800:	learn: 4.6725781	test: 4.3596467	best: 4.3596467 (800)	total: 14m 20s	remaining: 3m 33s
900:	learn: 4.6392664	test: 4.3581746	best: 4.3581688 (899)	total: 16m 8s	remaining: 1m 46s
999:	learn: 4.6087766	test: 4.3565541	best: 4.3565197 (997)	total: 17m 52s	remai

In [16]:
#model.save_model(f"model{model.learning_rate_}_{model.best_score_}.cbm")

In [17]:
#XX["target"].is_null().sum()
#X
model.get_feature_importance(prettified=True).head(60)

,Feature Id,Importances
0,day2,11.793920
1,dy2,9.977457
2,dy1,8.641744
3,imax1,8.294827
4,imax2,6.483974
5,imean,5.981443
6,imax3,4.635014
7,imax4,3.758862
8,imedian,3.359560
9,diff_datopen_report_date_flg,2.797257


In [18]:
model.get_feature_importance(prettified=True)[60:120]

,Feature Id,Importances
60,imax47,0.150213
61,imax45,0.147582
62,imax48,0.143885
63,imax41,0.141760
64,w7_1,0.122143
65,imax44,0.113261
66,w7_4,0.106453
67,w7_6,0.104096
68,imax40,0.104065
69,year2,0.078980


In [19]:
model.get_feature_importance(prettified=True).tail(60)

,Feature Id,Importances
25,w7,0.724791
26,id_region,0.717300
27,imax19,0.523611
28,imax25,0.508844
29,imax23,0.491585
30,imax24,0.478819
31,imax26,0.472209
32,imax18,0.468483
33,imax20,0.424269
34,imax50,0.399204


In [20]:
%%time
innweeks0 = pl.read_parquet("data/target_series.parquet"
                            ).with_columns(                              
                                pl.col("week").cast(pl.Int8),
                                np.log1p(pl.col("target")),
                            )
innweeks0

innweeks0 = innweeks0.with_columns(
                      pl.col("target").shift(0).over("inn_id").alias("imax1"),
                      pl.col("target").shift(1).over("inn_id").alias("imax2"),
                      pl.col("target").shift(2).over("inn_id").alias("imax3"),
                      pl.col("target").shift(3).over("inn_id").alias("imax4"),
                      pl.col("target").shift(4).over("inn_id").alias("imax5"),
                      pl.col("target").shift(5).over("inn_id").alias("imax6"),
                      pl.col("target").shift(6).over("inn_id").alias("imax7"),
                      pl.col("target").shift(7).over("inn_id").alias("imax8"),
                      pl.col("target").shift(8).over("inn_id").alias("imax9"),
                      pl.col("target").shift(9).over("inn_id").alias("imax10"),
                      pl.col("target").shift(10).over("inn_id").alias("imax11"),
                      pl.col("target").shift(11).over("inn_id").alias("imax12"),
                      pl.col("target").shift(12).over("inn_id").alias("imax13"),
                      pl.col("target").shift(13).over("inn_id").alias("imax14"),
                      pl.col("target").shift(14).over("inn_id").alias("imax15"),
                      pl.col("target").shift(15).over("inn_id").alias("imax16"),
                      pl.col("target").shift(16).over("inn_id").alias("imax17"),
                      pl.col("target").shift(17).over("inn_id").alias("imax18"),
                      pl.col("target").shift(18).over("inn_id").alias("imax19"),
                      pl.col("target").shift(19).over("inn_id").alias("imax20"),
                      pl.col("target").shift(20).over("inn_id").alias("imax21"),
                      pl.col("target").shift(21).over("inn_id").alias("imax22"),
                      pl.col("target").shift(22).over("inn_id").alias("imax23"),
                      pl.col("target").shift(23).over("inn_id").alias("imax24"),
                      pl.col("target").shift(24).over("inn_id").alias("imax25"),
                      pl.col("target").shift(25).over("inn_id").alias("imax26"),
                      pl.col("target").shift(26).over("inn_id").alias("imax27"),
                      pl.col("target").shift(27).over("inn_id").alias("imax28"),
                      pl.col("target").shift(28).over("inn_id").alias("imax29"),
                      pl.col("target").shift(29).over("inn_id").alias("imax30"),
                      pl.col("target").shift(30).over("inn_id").alias("imax31"),
                      pl.col("target").shift(31).over("inn_id").alias("imax32"),
                      pl.col("target").shift(32).over("inn_id").alias("imax33"),
                      pl.col("target").shift(33).over("inn_id").alias("imax34"),
                      pl.col("target").shift(34).over("inn_id").alias("imax35"),
                      pl.col("target").shift(35).over("inn_id").alias("imax36"),
                      pl.col("target").shift(36).over("inn_id").alias("imax37"),
                      pl.col("target").shift(37).over("inn_id").alias("imax38"),
                      pl.col("target").shift(38).over("inn_id").alias("imax39"),
                      pl.col("target").shift(39).over("inn_id").alias("imax40"),
                      pl.col("target").shift(40).over("inn_id").alias("imax41"),
                      pl.col("target").shift(41).over("inn_id").alias("imax42"),
                      pl.col("target").shift(42).over("inn_id").alias("imax43"),
                      pl.col("target").shift(43).over("inn_id").alias("imax44"),
                      pl.col("target").shift(44).over("inn_id").alias("imax45"),
                      pl.col("target").shift(45).over("inn_id").alias("imax46"),
                      pl.col("target").shift(46).over("inn_id").alias("imax47"),
                      pl.col("target").shift(47).over("inn_id").alias("imax48"),
                      pl.col("target").shift(48).over("inn_id").alias("imax49"),
                      pl.col("target").shift(49).over("inn_id").alias("imax50"),
                      pl.col("target").shift(50).over("inn_id").alias("imax51"),
                      pl.col("target").shift(51).over("inn_id").alias("imax52"),
                      pl.col("target").shift(52).over("inn_id").alias("imax53"),
                      pl.col("target").shift(53).over("inn_id").alias("imax54"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(55).over("inn_id").alias("imax55"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(56).over("inn_id").alias("imax56"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(57).over("inn_id").alias("imax57"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(58).over("inn_id").alias("imax58"),
                      #pl.col("target").rolling_max(window_size=1,min_periods=1).shift(59).over("inn_id").alias("imax59"),
                      
                      #pl.col("target").rolling_max(window_size=53,min_periods=1).shift(0).over("inn_id").alias("imax"),
                      #pl.col("target").rolling_min(window_size=53,min_periods=1).shift(0).over("inn_id").alias("imin"),
                      pl.col("target").rolling_mean(window_size=53,min_periods=1).shift(0).over("inn_id").alias("imean"),
                      pl.col("target").rolling_median(window_size=53,min_periods=1).shift(0).over("inn_id").alias("imedian"),
    
                      #pl.col("target").rolling_quantile(quantile=0.95, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq095"),
                      #pl.col("target").rolling_quantile(quantile=0.90, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq090"),
                      #pl.col("target").rolling_quantile(quantile=0.85, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq085"),
                      #pl.col("target").rolling_quantile(quantile=0.80, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq080"),
                      #pl.col("target").rolling_quantile(quantile=0.75, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq075"),
                      #pl.col("target").rolling_quantile(quantile=0.70, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq070"),
                      #pl.col("target").rolling_quantile(quantile=0.65, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq065"),
                      #pl.col("target").rolling_quantile(quantile=0.60, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq060"),
                      #pl.col("target").rolling_quantile(quantile=0.55, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq055"),
    
                      #pl.col("target").rolling_quantile(quantile=0.45, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq045"),
                      #pl.col("target").rolling_quantile(quantile=0.40, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq040"),
                      #pl.col("target").rolling_quantile(quantile=0.35, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq035"),
                      #pl.col("target").rolling_quantile(quantile=0.30, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq030"),
                      #pl.col("target").rolling_quantile(quantile=0.25, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq025"),
                      #pl.col("target").rolling_quantile(quantile=0.20, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq020"),
                      #pl.col("target").rolling_quantile(quantile=0.15, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq015"),
                      #pl.col("target").rolling_quantile(quantile=0.10, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq010"),
                      #pl.col("target").rolling_quantile(quantile=0.05, window_size=53,min_periods=1).shift(0).over("inn_id").alias("iq005"),
                     )
innweeks0 = innweeks0.group_by("inn_id").last().drop(["week","target"])
innweeks0

CPU times: user 11.4 s, sys: 7.42 s, total: 18.8 s
Wall time: 17.8 s


inn_id,imax1,imax2,imax3,imax4,imax5,imax6,imax7,imax8,imax9,imax10,imax11,imax12,imax13,imax14,imax15,imax16,imax17,imax18,imax19,imax20,imax21,imax22,imax23,imax24,imax25,imax26,imax27,imax28,imax29,imax30,imax31,imax32,imax33,imax34,imax35,imax36,imax37,imax38,imax39,imax40,imax41,imax42,imax43,imax44,imax45,imax46,imax47,imax48,imax49,imax50,imax51,imax52,imax53,imax54,imean,imedian
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""inn4003333""",13.285554,13.260007,13.475712,13.416035,13.436578,13.291825,13.123689,13.066825,13.222223,13.051713,12.497941,12.532931,12.183794,12.417227,10.483113,9.006113,0.0,9.683265,8.749089,9.355587,9.004819,9.895699,9.388522,9.411618,9.935353,10.274265,0.0,0.0,9.557306,9.976042,0.0,9.614953,0.0,10.2392,8.496826,9.204535,9.738079,0.0,9.712199,9.02766,10.332034,10.942409,11.451582,11.412231,11.425284,11.25348,11.212668,11.553329,11.456136,12.097345,12.02766,11.58429,12.426176,12.134947,9.852659,10.483113
"""inn4187507""",12.780411,14.717354,13.66155,13.009504,12.375038,13.07665,11.505284,13.009513,12.800059,10.614919,11.56049,12.554363,9.580032,0.0,12.767722,12.35654,11.883389,12.138313,12.397515,11.392348,12.037348,12.026882,12.80298,12.982182,13.433242,10.839044,9.37648,10.517892,9.424508,8.456982,0.0,11.460514,11.806319,12.161389,12.282827,11.136612,13.292263,9.937868,10.278817,11.871228,12.808386,13.981261,10.222808,11.44963,13.07588,11.044789,12.279826,12.559401,13.678503,11.766444,12.015061,11.213918,12.157493,12.4031,11.48226,12.037348
"""inn4674990""",11.830336,11.317058,9.622602,14.411154,14.258335,14.014478,13.080111,14.11175,13.372386,14.020855,13.020224,13.505734,14.43208,11.276606,13.092994,13.992728,14.165368,13.220029,13.404377,14.830439,11.936397,10.624948,15.431981,12.463973,13.331298,14.511381,11.748902,10.805973,15.46535,14.281752,0.0,13.519064,15.116284,14.069744,14.751589,14.493314,14.770246,12.780352,14.765598,12.230097,11.507895,13.03402,13.851971,13.732172,12.974929,12.475395,14.21778,13.84161,15.698722,13.272903,13.785866,13.509814,15.154352,15.191181,13.191232,13.519064
"""inn4952377""",12.465011,11.106061,11.540917,11.02487,9.936867,12.222575,10.644684,10.832117,6.84455,10.839643,9.855819,14.351088,7.825695,9.275294,11.662876,12.360733,11.65833,8.861589,13.512582,13.301141,13.065094,11.610586,13.426375,9.359248,10.308168,13.187094,12.592936,13.505993,11.277245,11.111178,8.477021,11.716381,13.462551,9.24218,9.632341,13.633419,10.824619,9.036616,10.939232,12.188099,10.564999,10.766971,8.3119,12.295929,12.45153,7.932643,7.687419,10.173145,12.873327,9.113091,12.617945,12.891374,11.88167,10.832441,11.099637,11.111178
"""inn1413808""",0.0,11.305308,11.61811,11.897565,10.819574,11.602704,11.940271,10.130857,9.013887,10.616686,13.996522,11.93122,0.0,11.661762,10.266168,13.954273,13.524009,11.927505,11.236139,11.240757,14.18555,12.290374,14.591134,14.809167,12.973895,0.0,13.49598,12.451691,13.90979,13.278122,0.0,13.626002,14.836732,11.131249,11.988056,13.750364,11.313877,11.683088,11.532269,9.850535,11.641782,13.74458,14.171425,13.550157,12.693602,13.037413,12.669589,13.511939,12.828014,0.0,12.738064,11.771667,12.235855,12.55124,11.225949,11.940271
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""inn473655""",13.956258,14.149284,13.419399,14.319089,12.574567,12.673297,14.259736,14.128776,12.715084,12.022542,14.607831,14.475995,11.994299,12.687152,14.957361,14.611726,12.984356,14.028055,12.582579,13.287563,13.612527,14.35983,14.129689,13.34487,13.976122,13.211151,14.729078,13.844455,0.0,14.437084,0.0,10.734053,14.442561,12.059503,12.591835,13.915393,14.043997,0.0,0.0,11.453416,13.185017,0.0,6.585313,8.882356,10.996794,11.00201,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.112679,12.715084
"""inn632650""",13.175553,0.0,8.884263,0.0,8.760369,1

In [21]:
X0=pl.read_csv("data/sample_submit.csv"
           ).with_columns( pl.col("week").cast(pl.Int8)
            ).join(weeks,
                   on="week",
                   how="left"
                  ).join(profiles,on="inn_id",
                         how="left"
                        ).join(innweeks0,on="inn_id",
                         how="left"
                               
                        ).select(X_train.columns).to_pandas()
X0

,inn_id,imax1,imax2,imax3,imax4,imax5,imax6,imax7,imax8,imax9,...,w7_1,w7_2,w7_3,w7_4,w7_5,w7_6,ipul,id_region,main_okved_group,diff_datopen_report_date_flg
0,inn1000051,15.821142,11.487807,14.610071,15.044252,11.130213,11.644807,10.373773,9.118413,13.542485,...,5,5,5,5,5,5,0,45,68,48.0
1,inn1000051,15.821142,11.487807,14.610071,15.044252,11.130213,11.644807,10.373773,9.118413,13.542485,...,5,5,5,5,5,5,0,45,68,48.0
2,inn1000051,15.821142,11.487807,14.610071,15.044252,11.130213,11.644807,10.373773,9.118413,13.542485,...,5,5,5,5,5,5,0,45,68,48.0
3,inn1000051,15.821142,11.487807,14.610071,15.044252,11.130213,11.644807,10.373773,9.118413,13.542485,...,5,5,5,5,5,5,0,45,68,48.0
4,inn1000051,15.821142,11.487807,14.610071,15.044252,11.130213,11.644807,10.373773,9.118413,13.542485,...,5,5,5,5,5,5,0,45,68,48.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623551,inn999886,16.799984,17.025046,16.736929,17.042798,15.637834,16.703016,16.853691,16.485220,16.193706,...,5,5,3,5,5,5,1,22,45,192.0
623552,inn999886,16.799984,17.025046,16.736929,17.042798,15.637834,16.703016,16.853691,16.485220,16.193706,...,5,5,5,5,5,5,1,22,45,192.0
623553,inn999886,16.799984,17.025046,16.736929,17.042798,15.637834,16.703016,16.853691,16.485220,16.193706,...,5,5,5,5,5,5,1,22,45,192.0
623554,inn999886,16.799984,17.025046,16.736929,17.042798,15.637834,16.703016,16.853691,16.485220,16.193706,...,5,5,5,5,5,5,1,22,45,192.0


In [22]:
#model.load_model("model0.15696200728416443_{'learn': {'RMSE': 2.1840243465326776}, 'validation': {'RMSE': 2.4295050367945095}}.cbm")
model

In [23]:
np.expm1( np.clip(model.predict(X0), 0, np.inf) )

array([[  403424.4497401 ,   235718.32449726,   365613.9969662 ,
          406499.33647845],
       [  370285.21502865,   211772.28592535,   310088.47094714,
          540873.88424923],
       [  285891.97809388,   181316.07781389,   322026.21105134,
          410276.91141872],
       ...,
       [17981763.06861388, 18082872.9964847 , 19006890.36781559,
        15540877.24095633],
       [18938244.66536409, 18729338.19092793, 18555498.71832491,
        17076131.45602413],
       [18625256.93095101, 18799366.48183089, 18172430.97049966,
        16816581.06738651]])

In [24]:
Xp=pl.read_csv("data/sample_submit.csv"
           ).to_pandas()
pp=np.expm1( np.clip(model.predict(X0), 0, np.inf) )
Xp["predict"]=np.ravel(np.concatenate((pp, pp*0, pp*0), axis=1)[::12,])
Xp.to_csv("pred_07.csv")
Xp

,inn_id,week,predict
0,inn1000051,106,403424.449740
1,inn1000051,107,235718.324497
2,inn1000051,108,365613.996966
3,inn1000051,109,406499.336478
4,inn1000051,110,0.000000
...,...,...,...
623551,inn999886,113,0.000000
623552,inn999886,114,0.000000
623553,inn999886,115,0.000000
623554,inn999886,116,0.000000
